In [1]:
#Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import spacy
nlp = spacy.load('en_core_web_sm')
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
#Read the datasets
amazon_data = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)
imdb_data = pd.read_csv('imdb_labelled.txt', sep='\t', header=None)
yelp_data = pd.read_csv('yelp_labelled.txt', sep='\t', header=None)

In [3]:
#Display data info
amazon_data.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [4]:
imdb_data.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [5]:
yelp_data.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [6]:
#Assign headers
columns = ['Review', 'Sentiment']
amazon_data.columns = columns
imdb_data.columns = columns
yelp_data.columns = columns

print(amazon_data.shape)
print(imdb_data.shape)
print(yelp_data.shape)

(1000, 2)
(748, 2)
(1000, 2)


In [7]:
#Combine all datasets in one
data = amazon_data.append([imdb_data, yelp_data], ignore_index=True)
print(data.head())
print(data.shape)

                                              Review  Sentiment
0  So there is no way for me to plug it in here i...          0
1                        Good case, Excellent value.          1
2                             Great for the jawbone.          1
3  Tied to charger for conversations lasting more...          0
4                                  The mic is great.          1
(2748, 2)


In [8]:
#Check Sentiment distribution(positive=1,negative=0)
print(data['Sentiment'].value_counts())
print()
#Check for missing values
print(data.isnull().sum())

x = data['Review']
y = data['Sentiment']

1    1386
0    1362
Name: Sentiment, dtype: int64

Review       0
Sentiment    0
dtype: int64


In [9]:
#Data Preprocessing

punct = string.punctuation #all punctuation
stopwords = list(STOP_WORDS) #all stopwords

def data_cleaning(utterance):
  #load spacy model
  doc = nlp(utterance)

  tokens = []
  for token in doc:
    '''if the lemma of the token isn't a pronoun, it's lowercased
       if the word itself is a pronoun, it's lowercased immediately as there's 
       no lemma'''
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
  
  data = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      data.append(token)
  
  return data

In [10]:
data_cleaning("Hello! This is a cleaned version of the sentence I'm adding now...")

['hello', 'clean', 'version', 'sentence', 'add', '...']

In [11]:
#Feature Engineering(Vectorization)
#TF-IDF Application
tfidf = TfidfVectorizer(tokenizer=data_cleaning) #tokenization based on the particular function

#Model Creation
svm = LinearSVC()

#Data Splitting
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

print(x_train.shape, x_test.shape)

(2198,) (550,)


In [12]:
#Model Training
classifier = Pipeline([('tfidf',tfidf), ('svm', svm)]) #first perform vectorization and then training
model = classifier.fit(x_train,y_train)

In [13]:
#Predictions
y_pred = model.predict(x_test)
#Metrics
accuracy = accuracy_score(y_test, y_pred)
confusion_matrix = confusion_matrix(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)

In [14]:
print(accuracy*100)
print()
print(confusion_matrix)
print()
print(classification_report)

76.9090909090909

[[215  66]
 [ 61 208]]

              precision    recall  f1-score   support

           0       0.78      0.77      0.77       281
           1       0.76      0.77      0.77       269

    accuracy                           0.77       550
   macro avg       0.77      0.77      0.77       550
weighted avg       0.77      0.77      0.77       550



In [15]:
model.predict(["I like using the tool of the website. It's helpful enough!"])

#output=1 -> positive review

array([1])

In [16]:
model.predict(["I don't like using this tool. It's not helpful and it's very slow!!"])

#output=0 -> negative review

array([0])